<a href="https://colab.research.google.com/github/machren/NNQuantumStates/blob/main/QIC_TFI_DMRG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Installing requirements for tenpy

In [10]:
!git clone https://github.com/tenpy/tenpy.git

fatal: destination path 'tenpy' already exists and is not an empty directory.


In [11]:
%cd tenpy

/content/tenpy/tenpy


In [12]:
!pip install -e .

Obtaining file:///content/tenpy/tenpy
ERROR: file:///content/tenpy/tenpy does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [13]:
import tenpy
print(tenpy.__version__)


1.0.5


### Transverse Field Ising Model and DMRG in TeNPy

Here we describe implementation of the transverse field Ising model (TFIModel) using the Tensor Network Python library (TeNPy), focusing on its Hamiltonian structure, model parameters, DMRG algorithm settings, and the theoretical background based on the matrix product state (MPS) framework introduced in [Schollwöck (2011)](https://arxiv.org/abs/1008.3477).

---

#### Transverse Field Ising Model

The TFIM is defined on a lattice with the Hamiltonian consisting of nearest-neighbor spin interactions and a transverse magnetic field. The model captures the competition between ordering (due to spin-spin coupling) and quantum fluctuations (due to the transverse field).

#### Model Parameters

The model is specified via a configuration dictionary. The values in the example are selected for physical and numerical reasons:

- **`L = N`** – System size, defining the number of lattice sites. Typical choices range from 10 to 100 depending on available computational resources. Larger sizes approach the thermodynamic limit but increase computational cost.

- **`J = 1.0`** – The spin-spin coupling constant. Setting \( J = 1 \) is a standard normalization choice that allows comparison across different simulations. Other parameters (like `g`) are then interpreted relative to this unit.

- **`g = 1.0`** – The strength of the transverse magnetic field. The value \( g = 1 \) is particularly significant, as it represents the critical point of the 1D transverse field Ising model in the thermodynamic limit. At this value, the system undergoes a quantum phase transition from a ferromagnetic to a paramagnetic phase.

- **`bc_MPS = 'finite'`** – Boundary conditions applied to the MPS representation. 'Finite' implies open boundary conditions, suitable for systems of finite size. They are easier to handle numerically than periodic conditions.

---

#### DMRG Algorithm Parameters

TeNPy’s DMRG implementation is highly customizable. The parameters selected in the example balance precision, convergence reliability, and computational efficiency.

#### Algorithm Parameters

- **`mixer = True`** – Enables the use of a mixer, which introduces noise into the local optimization steps during early sweeps. This helps prevent the algorithm from getting stuck in local minima. It is gradually turned off, allowing fine convergence near the true ground state.

- **`max_E_err = 1e-10`** – Convergence threshold for the energy. The DMRG iterations stop when the energy changes between sweeps fall below this value. The choice of \(10^{-10}\) ensures highly accurate energy estimates suitable for precision studies, albeit with longer computation times.

- **`trunc_params`**:
  - **`chi_max = 1000`** – The maximum number of states (bond dimension) retained during truncation. A value of 1000 strikes a balance between accuracy and memory usage. Larger `chi_max` allows for more entanglement to be captured but increases computational cost.
  - **`svd_min = 1e-10`** – Minimum singular value threshold. Singular values smaller than this are discarded. This cutoff ensures numerical stability and avoids keeping noise-level components in the MPS.

- **`verbose = 1`** – Controls the level of output. A value of 1 provides moderate logging of DMRG progress without overwhelming detail, useful for monitoring convergence in typical simulations.

- **`N_sweeps_check = 2`** – The number of sweeps between convergence checks. Frequent checks help terminate the algorithm promptly when convergence is reached, saving computational time.

---

#### DMRG Implementation in TeNPy

TeNPy implements the Density Matrix Renormalization Group (DMRG) based on the modern formulation using Matrix Product States, as developed in [Schollwöck (2011)](https://arxiv.org/abs/1008.3477).

#### Key Features of the MPS-Based DMRG Algorithm

1. **Matrix Product States (MPS)**  
   Quantum many-body wavefunctions are approximated as a product of local tensors. This efficiently encodes entanglement for gapped 1D systems, leveraging the area law.

2. **Variational Optimization**  
   DMRG iteratively optimizes the MPS by minimizing the system's energy, sweeping back and forth over the chain. At each step, tensors are updated locally based on their effective Hamiltonian.

3. **Two-Site and One-Site Updates**  
   The algorithm typically begins with two-site updates to ensure robustness and transitions to faster one-site updates as convergence improves. The transition is guided by convergence metrics and the `mixer` mechanism.

4. **Truncation and SVD**  
   After optimization, tensors are decomposed using Singular Value Decomposition. The MPS is truncated to retain only the most significant singular values (based on `chi_max` and `svd_min`) to control the growth of computational complexity.

5. **Convergence and Stopping Criteria**  
   Energy convergence is monitored based on `max_E_err` and checked every `N_sweeps_check` sweeps. Once changes in energy become negligible, the algorithm halts, yielding the ground-state MPS.

---

## References

- Schollwöck, U. (2011). *The density-matrix renormalization group in the age of matrix product states*. [arXiv:1008.3477](https://arxiv.org/abs/1008.3477)


In [23]:
import numpy as np
import time
from tenpy.models import TFIModel
from tenpy.networks.mps import MPS
from tenpy.algorithms import dmrg

In [24]:
# DMRG parameters
dmrg_params = {
    'mixer': True,            # to avoid that the algorithm gets stuck in local energy minima
    'max_E_err': 1e-10,
    'trunc_params': {
        'chi_max': 1000,
        'svd_min': 1e-10,
    },
    'verbose': 1,
    'N_sweeps_check': 2,
}

ground_energies = []
times = []

In [25]:
for N in range(10, 100, 10):
  # Define model parameters
  model_params = {
      'L': N,                   # system size
      'J': 1.0,                 # coupling constant
      'g': 1.0,                 # transverse field
      'bc_MPS': 'finite',       # boundary conditions
  }

  # Initialize the model
  model = TFIModel(model_params)

  # Build the initial product state (all spins up)
  product_state = ["up"] * model.lat.N_sites

  psi = MPS.from_product_state(model.lat.mps_sites(), product_state, bc=model_params['bc_MPS'])

  # Run the DMRG algorithm
  start_time = time.time()
  info = dmrg.run(psi, model, dmrg_params)
  end_time = time.time()

  times.append(end_time - start_time)
  ground_energies.append(info['E'])

  print(f"N = {N}: Ground state energy = {info['E']}")
  print(f"Time taken is {end_time - start_time} seconds")

/content/tenpy/tenpy/tools/params.py:232: UserWarning: unused option ['verbose'] for config DMRG
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


N = 10: Ground state energy = -12.381489999654743
Time taken is 1.7437689304351807 seconds


/content/tenpy/tenpy/tools/params.py:232: UserWarning: unused option ['verbose'] for config DMRG
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


N = 20: Ground state energy = -25.107797111623743
Time taken is 6.577810525894165 seconds


/content/tenpy/tenpy/tools/params.py:232: UserWarning: unused option ['verbose'] for config DMRG
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


...

/content/tenpy/tenpy/tools/params.py:232: UserWarning: unused option ['verbose'] for config DMRG
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


N = 60: Ground state energy = -76.03315613032177
Time taken is 103.88608288764954 seconds


/content/tenpy/tenpy/tools/params.py:232: UserWarning: unused option ['verbose'] for config DMRG
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


N = 70: Ground state energy = -88.76524466396042
Time taken is 165.7413694858551 seconds


KeyboardInterrupt: 

In [ ]:
# Print ground state energy
print("Ground state energy:", ground_energies)

In [ ]:
# Optional: print entanglement entropy
S = psi.entanglement_entropy()
print("Entanglement entropy:", S)